<a href="https://colab.research.google.com/github/hindhammad24/Computer-Vision-Project/blob/main/computervisionproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Plantify** -  Plant Health Detector


**By Hind Hammad and Manmeet Sagri**

**Clasification**: Assigns one label to the whole image for example "This plant is diseased.”


**Project Overview**

This project, Plantify or Plant Health Detector, applies computer vision to classify plant images as healthy or diseased.
The goal is to help everyday gardeners identify early signs of plant disease before visible damage becomes severe.
By fine-tuning a YOLOv8 classification model, we aim to train a system that recognizes general plant health from photos taken in outdoor garden conditions.

**Problem**

Many home gardeners struggle to detect early plant stress or infection, which often leads to preventable plant loss.
Our model analyzes plant photos and classifies them into two categories, Healthy and Diseased, to support better plant care and timely action.

**Users**

The primary users are home gardeners and plant enthusiasts who want a quick, AI-based way to check the health of their plants.
In practice, this could be integrated into a mobile or web app that allows users to upload or capture plant images and instantly get feedback on their health condition.

**Feedback and Revisions**

After receiving feedback from Dr. Yalcin and our classmates, we refined our project to make it more focused and realistic. We chose to concentrate on a smaller set of common outdoor garden plants because they are widely grown and show clear visual signs of health or disease. Our model now classifies only two conditions, healthy or diseased, regardless of plant type. This change keeps the project manageable and ensures that Plantify remains practical and useful for home and community gardeners who care for outdoor plants.


**Issues and Biases in Data Collection**

While collecting and combining data, we encountered several minor issues:

- **Misplaced images:** Some images were incorrectly categorized (e.g., healthy leaves inside diseased folders). We manually reviewed and corrected these errors.

- **Class imbalance:** The dataset contained more diseased images (≈63%) than healthy ones (≈37%). To reduce this bias, we oversampled healthy images during training and applied data augmentation techniques such as flipping, rotation, and color adjustments.

- **Lighting and background variation**: While most Kaggle images had uniform backgrounds. To improve consistency, we ensured diverse conditions were included in both classes.

We addressed these issues through careful verification, balancing, and augmentation, making the dataset more representative of real-world garden conditions.

# **Connect to google Drive**

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The initial step connects the Colab notebook to Google Drive, allowing both of us (Hind and Manmeet) to access and save files directly from our shared Drive folder.
This ensures the dataset and training outputs are automatically saved and synchronized. This step allows Colab to read image files from Drive, run augmentation and training scripts, and then save the processed data and results back into Drive automatically.
We confirmed that our dataset folders, Healthy and Diseased, were correctly located inside the main project directory Plantify.
This connection ensures our files stay safe and synced, and we can easily resume work later without losing progress..

In [8]:
import os, glob

ROOT = "/content/drive/MyDrive/plantify"   # This folder contains: Healthy/ and Diseased/
HEALTHY_DIR = f"{ROOT}/Healthy"
DISEASED_DIR = f"{ROOT}/Diseased"

# Quick counts
h_files = glob.glob(f"{HEALTHY_DIR}/*")
d_files = glob.glob(f"{DISEASED_DIR}/*")
print("Healthy:", len(h_files))
print("Diseased:", len(d_files))

Healthy: 8989
Diseased: 15328


We first counted how many images were in each class folder, Healthy and Diseased, to understand the class balance before training.
We found that the Healthy class had 8,981 images, while the Diseased class had 15,276 images, meaning the dataset was unbalanced.
This imbalance can make the model biased toward predicting diseased plants more often, so we planned to fix it through augmentation adding more Healthy samples artificially.

# **Clone YOLOv8 Repository**

In [9]:
# Step 1: We cloned and installed YOLOv8 from the official Ultralytics GitHub repository


# Reset to root and recreate /content directory since the first time it gave us error
%cd /
!mkdir -p /content
%cd /content

# Remove any previous YOLO installation
!rm -rf ultralytics

# Clone
!git clone https://github.com/ultralytics/ultralytics.git

# Install YOLOv8 in editable mode
!python -m pip install -q -e /content/ultralytics

# Move into the YOLO folder
%cd /content/ultralytics

# Import YOLO
import ultralytics, os, glob, shutil, random
from pathlib import Path

print("✅ YOLO installed successfully and environment ready.")


/
/content
Cloning into 'ultralytics'...
remote: Enumerating objects: 71797, done.
remote: Counting objects: 100% (615/615), done.
remote: Compressing objects: 100% (341/341), done.
remote: Total 71797 (delta 471), reused 281 (delta 274), pack-reused 71182 (from 3)
Receiving objects: 100% (71797/71797), 38.72 MiB | 25.08 MiB/s, done.
Resolving deltas: 100% (53765/53765), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
/content/ultralytics
✅ YOLO installed successfully and environment ready.


In this initial step, we installed and cloned the official YOLOv8 repository from Ultralytics. This gives us access to the latest version of the YOLO framework, which supports image classification as per toturials.

By cloning directly from GitHub, we ensure that our project follows the course instructions and uses the exact implementation specified by the professor.
Installing the repository in editable mode (-e) allows us to modify or reload YOLO files if needed during debugging or experimentation.

# **Dataset Description**





Our dataset, titled Plantify, was created by combining images from two open-source datasets available on Kaggle:

- **PlantVillage Dataset** — https://www.kaggle.com/datasets/emmarex/plantdisease

- **New Plant Diseases Dataset** — https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

In addition to these sources, we each contributed 30 original images (60 total) that we captured ourselves to improve diversity, particularly for garden plants photographed outdoors in natural lighting.

The combined dataset contains images of common garden plants such as apple, tomato, grape, corn, pepper, peach, and mint, categorized into two main classes:

- **Healthy**

- **Diseased**

Each image shows a single plant leaf photographed under natural or controlled lighting conditions. The dataset was reorganized by us to focus on plant health rather than plant type.

Healthy images: **8,981**

Diseased images: **15,276**

Total (before our additions): **24,257** images

New total after adding 60 images: **24,317** images

## **Naming Convention**

Each image filename follows a consistent structure that identifies both the plant and its condition:

**plant_condition_number.jpg**


Examples:

- *apple_healthy_1.jpg*

- *peach_bacterialspot_1.jpg*

This naming system ensures that the model can easily recognize each class label from the folder structure, while filenames remain traceable and easy to organize.

**Hind Hammad’s Contribution**

Hind added 30 new images:

- 8 Healthy images

- 22 Diseased images

**plant_condition_hind_number.jpg**


Examples:

- *basil_healthy_hind_3.jpg*

- *mint_diseased_hind_5.jpg*

These images were captured in outdoor settings with different lighting conditions to help the model generalize better to real-world use cases.

**Manmeet Sagri’s Contribution**

Manmeet added a total of 30 images to the dataset:

- 12 Healthy images

- 18 Diseased images

All her images follow the same structured naming convention with an added identifier to indicate the contributor:

**plant_condition_contributor_number.jpg**


Examples:

- *tomatoes_healthy_manmeet_4.jpg*

- *parsley_diseased_manmeet_1.jpg*

This convention helps maintain dataset consistency and allows us to identify contributor-specific samples for tracking and quality control.

**Annotations**

For our project, manual annotation using external tools such as Label Studio or MakeSense.ai was not required, as our task involves image classification rather than object detection.
We defined two labels , Healthy and Diseased, and placed each image in its respective folder. This folder-based structure automatically serves as the annotation for YOLO classification training.
Consistent file naming conventions further ensured labeling accuracy and traceability during dataset preparation.



# **File Structure Setup**

After cleaning and organizing all images into the two main folders, Healthy and Diseased, our next step was to format the dataset into the folder hierarchy accepted by YOLOv8 classification.
Since YOLO reads class labels directly from folder names, we created two subfolders inside both the train and validation directories, one for each class. The final folder structure looks like this:

**PLEASE DON'T TRY RUNNING THIS CODE IT IS JUST TO SHOW OUR FILE STRUCTURE**

In [ ]:
Plantify/
├── train/
│   ├── Healthy/
│   │   ├── apple_healthy_1.jpg
│   │   ├── tomato_healthy_2.jpg
│   │   └── ...
│   └── Diseased/
│       ├── apple_rust_1.jpg
│       ├── peach_bacterialspot_2.jpg
│       └── ...
└── val/
    ├── Healthy/
    │   ├── pepper_healthy_1.jpg
    │   └── basil_healthy_2.jpg
    └── Diseased/
        ├── tomato_leafmold_1.jpg
        └── grape_blackrot_2.jpg


This structure allows YOLO to automatically recognize “Healthy” and “Diseased” as the two class labels during training.
We used a 2/3 to 1/3 split, 67% of images for training and 33% for validation, following the course guideline that specifies two-thirds for training and the rest for evaluation.
This ensures that the model learns from most of the data but is still tested on unseen samples for fair performance evaluation.

# **Data Preprocessing**



Before training, we performed several basic preprocessing steps to ensure data consistency and model compatibility:

- Removed misplaced images: A few healthy images were found inside the diseased folder (and vice versa). We corrected these manually.

- Verified dataset integrity: We checked that every image file could open successfully and that there were no missing or unreadable files.

- Standardized image naming: Using PowerShell, we renamed all files with the format plant_condition_number (e.g., apple_healthy_1.jpg) to avoid issues caused by spaces or parentheses.

- Folder-based labeling: Because this is a classification task, we did not need to create .txt annotation files. The folder names (“Healthy” and “Diseased”) act as labels, which YOLO automatically reads during training.

These preprocessing steps ensured that our dataset was clean, uniform, and directly compatible with YOLO classification without requiring any extra label files.

# **Split files**

In [10]:
# Step 2: Fast 2/3 split for YOLO dataset
# using SYMLINKS instead of copying

# In our project, we first tried copying all images from our Drive folders,
# but it took hours because Google Drive is very slow with large datasets.
# So we switched to creating SYMLINKS or shortcuts instead of
# physically copying files. This keeps the same folder structure but
# runs much faster, finishing in less than a minute even for 20k+ images.

from pathlib import Path
import os, random, shutil

# Our dataset location
# We kept our original Plantify dataset inside Google Drive
ROOT = Path("/content/drive/MyDrive/plantify")
CLASSES = ["Healthy", "Diseased"]  # class names

# Instead of putting it back on Drive
# we save it locally in Colab's /content directory.
DST = Path("/content/Plantify_YOLO")

# Before creating new links, we clear any old split to start fresh since we initially had trouble with this
if DST.exists():
    shutil.rmtree(DST)

# Ratio and random seed setup
# We chose to use a 2/3 : 1/3 train validation split as per the instructions
TRAIN_RATIO = 2/3
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# YOLO expects a {train,val}/{Healthy,Diseased}
for split in ["train", "val"]:
    for c in CLASSES:
        (DST / split / c).mkdir(parents=True, exist_ok=True)

# Function to list all image files in each class folder
# This helps us grab only actual image files and ignore others just incase there is other files
def list_images(dir_path: Path):
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
    return [p for p in dir_path.iterdir() if p.is_file() and p.suffix.lower() in exts]

# Function to create a symlink
# A symlink points to the original file, so no data is duplicated.
# If the link already exists, we skip it to save time.
def safe_symlink(src: Path, dst: Path) -> bool:
    if dst.exists() or dst.is_symlink():
        return False
    os.symlink(src.resolve(), dst)
    return True

# Function to split and link files for each class
# Here, we shuffle images so training is randomized, split them by ratio,
# and then create symlinks to the original files in the correct train/val folders.
def split_and_link_class(src_dir: Path, class_name: str, train_ratio: float = TRAIN_RATIO):
    files = list_images(src_dir)
    random.shuffle(files)
    n_total = len(files)
    n_train = int(round(n_total * train_ratio))

    train_files = files[:n_train]
    val_files   = files[n_train:]

    tdir = DST / "train" / class_name
    vdir = DST / "val"   / class_name

    made_train = sum(safe_symlink(p, tdir / p.name) for p in train_files)
    made_val   = sum(safe_symlink(p, vdir / p.name) for p in val_files)

    print(f"{class_name:8s} -> train: {made_train:5d} | val: {made_val:5d} | total: {n_total:5d}")

# Run the split for both classes
for cls in CLASSES:
    src = ROOT / cls
    if not src.exists():
        raise FileNotFoundError(f"Missing class folder: {src}")
    split_and_link_class(src, cls)

print(" Symlink split complete at:", DST)

# Quick check of the final counts
# This gives us a summary of how many files were linked into train vs val.
def count_files(folder: Path) -> int:
    return sum(1 for p in folder.iterdir() if p.is_file())

train_counts = {c: count_files(DST / "train" / c) for c in CLASSES}
val_counts   = {c: count_files(DST / "val"   / c) for c in CLASSES}
total_train  = sum(train_counts.values())
total_val    = sum(val_counts.values())
ratio_train  = total_train / max(1, (total_train + total_val))


print("Train counts:", train_counts)
print("Val counts  :", val_counts)
print(f"YOLO-ready dataset at: {DST}")


Healthy  -> train:  5993 | val:  2996 | total:  8989
Diseased -> train: 10219 | val:  5109 | total: 15328
 Symlink split complete at: /content/Plantify_YOLO
Train counts: {'Healthy': 5993, 'Diseased': 10219}
Val counts  : {'Healthy': 2996, 'Diseased': 5109}
YOLO-ready dataset at: /content/Plantify_YOLO


**Explanation for what we did and why.**

At first, we used the old method of copying images from Google Drive into new train and validation folders using shutil.copy. However, this took a very long time because our dataset has over 20,000 images, and Drive is slow when handling many small files. Our first run took more than 53 minutes and we asked chatgpt how long we expect it to take it said more than 1 to 3+ hours.

After researching how to make this process faster, we discovered that symlinks symbolic links can be used to create lightweight pointers to existing files instead of duplicating them. This allowed us to organize our dataset into the YOLO format instantly without copying or moving any images.

We asked ChatGPT for help to guide us through this process, debug errors, and adapt the method to work inside Google Colab. During our research, we also found useful explanations and discussions from:

- GitHub issue on Jupytext and symlinks - https://github.com/mwouts/jupytext/issues/696


- Unix StackExchange post about what symlinking is and how it works - https://unix.stackexchange.com/questions/252561/what-is-a-symbolic-link-made-from-understanding-the-structure-of-symlinks


This new method made our workflow much easier, faster, and more efficient. We learned the difference between copying and linking files, and now our dataset setup runs in seconds instead of hours, while still following YOLO's required structure.

# **Train with YOLO’s built-in augmentation** BONUS

## Verify dataset paths

In [ ]:
# Step 3: Verify dataset paths
# We verify that our new YOLO-ready split exists so training won't break this is just becouse we had trouble in our initial run so this is us debugging in our own way
# Directory layout expected:
#   /.../Plantify/data/Plantify_YOLO/{train,val}/{Healthy,Diseased}

from pathlib import Path

DATA_DIR = " ... "

required = [
    f"{DATA_DIR}/train/Healthy",
    f"{DATA_DIR}/train/Diseased",
    f"{DATA_DIR}/val/Healthy",
    f"{DATA_DIR}/val/Diseased",
]
for p in required:
    assert Path(p).exists(), f"Missing: {p}"

print("✅ Dataset structure verified.")


Why we did this to catch path mistakes early and save time as part of our debugging since this already happened before.

## Train baseline YOLOv8 classification model with built-in augmentation for **BONUS**

In [ ]:
# Step 4: Train baseline model
# We start with the small 'n' backbone for a quick baseline.
# We enable YOLO's built-in augmentation to improve generalization BONUS.

from ultralytics import YOLO

BASE_RUN_NAME = "cls_baseline"
PROJECT_DIR   = " ... "  # where YOLO saves outputs

model = YOLO("yolov8n-cls.pt")  # pre-trained classifier

results = model.train(
    data=DATA_DIR,                # YOLO will infer classes from folder names
    epochs=15,                    # short run to get a baseline
    imgsz=224,                    # typical size for 'n' backbone
    batch=64,
    lr0=0.01,                     # default starting LR
    augment=True,                 # flips/rotations/color jitter built in
    project=PROJECT_DIR,
    name=BASE_RUN_NAME
)

print("✅ Baseline training finished.")

we wanted a fast baseline to measure initial performance before tuning.

## Evaluate baseline & collect core metrics

In [ ]:
# Step 5 : Evaluate baseline model
# We run validation to get top1/top5 accuracy and loss, and to
# generate confusion matrices for our report.

metrics = model.val(split="val")  # uses same data split
print("Top-1 Accuracy:", getattr(metrics, "top1", "N/A"))
print("Top-5 Accuracy:", getattr(metrics, "top5", "N/A"))
print("✅ Validation complete.")

to quantify performance and generate confusion matrix images we’ll include in the report.

## View confusion matrix

In [ ]:
# Step 6: Open confusion matrix
# We load the normalized confusion matrix image that YOLO saved.
# We'll paste this into the PDF and discuss error patterns.

from PIL import Image
from pathlib import Path

cm_path = Path(PROJECT_DIR) / BASE_RUN_NAME / "confusion_matrix_normalized.png"
if not cm_path.exists():

    cm_path = Path(PROJECT_DIR) / BASE_RUN_NAME / "confusion_matrix.png"

assert cm_path.exists(), "Confusion matrix image not found." #incase not found for debugging
display(Image.open(cm_path))
print("✅ Displayed confusion matrix:", cm_path)


Our observation to include in report ...

## Visualize predictions on validation images

In [ ]:
# Step 7 : Visualize predictions
# We generate prediction images to visually inspect correct vs. failed cases.
# Saved images are good evidence for the report.

pred_results = model.predict(
    source=f"{DATA_DIR}/val",     # run on validation set
    save=True,                    # save annotated predictions
    imgsz=224,
    project=PROJECT_DIR,
    name=f"{BASE_RUN_NAME}_preds"
)

print("✅ Saved prediction images to:", Path(PROJECT_DIR) / f"{BASE_RUN_NAME}_preds")

Why we did this to understand failure modes glare, shadow, clutter, subtle discoloration, etc.

## Detailed classification report

In [ ]:
# Step 8: Per class precision sklearn
# YOLO already reports accuracy, but we also want per-class PR/F1
# for a clearer bias picture Healthy vs. Diseased.

import glob, numpy as np
from sklearn.metrics import classification_report

# YOLO sorts class names alphabetically by folder name
class_names = ["Diseased", "Healthy"]

# Collect a sample from validation images for quick report
val_files, true_labels = [], []
for i, cls in enumerate(class_names):
    files = glob.glob(f"{DATA_DIR}/val/{cls}/*")
    val_files.extend(files)
    true_labels.extend([i]*len(files))

# Sample limit size to speed up and becouse the laptop is dieing
N = min(1000, len(val_files))
idxs = np.random.choice(len(val_files), size=N, replace=False)

pred_labels = []
for i in idxs:
    r = model.predict(source=val_files[i], save=False, verbose=False, imgsz=224)[0]
    pred = int(np.argmax(r.probs.data.cpu().numpy()))
    pred_labels.append(pred)

print(classification_report([true_labels[i] for i in idxs], pred_labels, target_names=class_names, digits=4))
print("✅ Generated per-class report.")


Why we did this to quantify per-class performance and show rubric-friendly metrics.

## Save example worked and failed images

In [ ]:
# Step 9: Save examples of correct/incorrect predictions
# We copy a few examples into 'examples/correct' and 'examples/incorrect'
# for the report’s qualitative analysis section.

from pathlib import Path
import shutil

examples_dir = Path(PROJECT_DIR) / BASE_RUN_NAME / "examples"
(examples_dir / "correct").mkdir(parents=True, exist_ok=True)
(examples_dir / "incorrect").mkdir(parents=True, exist_ok=True)

saved_correct = saved_incorrect = 0
limit = 6  # how many of each to keep

for i in idxs[:500]:  # scan a subset
    img_path = Path(val_files[i])
    r = model.predict(source=str(img_path), save=False, verbose=False, imgsz=224)[0]
    pred = int(np.argmax(r.probs.data.cpu().numpy()))
    true = 0 if "Diseased" in str(img_path) else 1

    if pred == true and saved_correct < limit:
        shutil.copy(str(img_path), examples_dir / "correct" / img_path.name)
        saved_correct += 1
    elif pred != true and saved_incorrect < limit:
        shutil.copy(str(img_path), examples_dir / "incorrect" / img_path.name)
        saved_incorrect += 1

    if saved_correct >= limit and saved_incorrect >= limit:
        break

print(f"✅ Saved {saved_correct} correct and {saved_incorrect} incorrect examples to:", examples_dir)


Why we did this to include concrete visuals in the PDF and discuss likely causes lighting, texture similarity, background clutter.

## Hyperparameter change required by rubric

In [ ]:
# Step 10: Train a tuned variant (hyperparameter change as per requirment)
# We increase image size and add dropout + mixup/cutmix to improve
# generalization and reduce bias toward 'Diseased'.

TUNED_RUN_NAME = "cls_hparam_change"

model_tuned = YOLO("yolov8n-cls.pt")

results_tuned = model_tuned.train(
    data=DATA_DIR,
    epochs=20,          # a bit longer
    imgsz=256,          # larger input for more detail but took forever to process and laptop died
    batch=64,
    lr0=0.01,
    dropout=0.05,       # regularization
    mixup=0.10,         # stronger augmentation
    cutmix=0.20,        # stronger augmentation
    augment=True,
    project=PROJECT_DIR,
    name=TUNED_RUN_NAME
)

print("✅ Tuned training finished.")


Why we did this to test if larger inputs and stronger regularization help the model reduce false Healthy → Diseased and improve robustness.

## Evaluate tuned model and compare

In [ ]:
# Step 11 : Evaluate tuned model & compare with baseline
# We compare top-1, curves, and confusion matrices across runs to
# show the impact of our hyperparameter change.

metrics_tuned = model_tuned.val(split="val")
print("Baseline top-1:", getattr(metrics, "top1", "N/A"))
print("Tuned    top-1:", getattr(metrics_tuned, "top1", "N/A"))

print("✅ Comparison ready. Grab results.png and confusion_matrix*.png from:")
print(" - Baseline:", Path(PROJECT_DIR) / BASE_RUN_NAME)
print(" - Tuned   :", Path(PROJECT_DIR) / TUNED_RUN_NAME)


What we saw tuned run generally improved top-1 and Healthy-class recall, with fewer false positives on Healthy images. ...

## Short, ready-to-paste comparison table to be added to report

In [ ]:
# Step 12: Print a tiny comparison summary for our report
# We print a quick, copy-pasteable summary table of top-1 accuracy.

b_top1 = getattr(metrics, "top1", None)
t_top1 = getattr(metrics_tuned, "top1", None)

print("\nModel Comparison (copy into report):")
print("| Run                | imgsz | dropout | mixup | cutmix | Top-1 |")
print("|--------------------|-------|---------|-------|--------|-------|")
print(f"| {BASE_RUN_NAME:18s} |  224  |  0.00   | 0.00  |  0.00  | {b_top1 if b_top1 is not None else 'N/A'} |")
print(f"| {TUNED_RUN_NAME:18s} |  256  |  0.05   | 0.10  |  0.20  | {t_top1 if t_top1 is not None else 'N/A'} |")

Why we did this the rubric asks to change one hyperparameter and compare; this gives a clean table to paste.